# Imports Necessários

In [15]:
#import os
import locale
import numpy as np
import pandas as pd
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
from gestao import ConectarAccess

# Definição de Variáveis e pré produção

In [16]:
# Informações de base de dados
tabela_cartao = '[historico_cartao]'
tabela_cc = '[extrato_cc]'
tabela_previsao = '[previsao_lancamento]'
arquivo_base = 'GestaoFinanceira1'

print(f">> Tabela Com o Historico do Cartão: {tabela_cartao}")
print(f">> Tabela Com o Extrato da Conta Corrente: {tabela_cc}")
print(f">> Tabela Para Ser Inserido os Lançamentos previsto: {tabela_previsao}")

>> Tabela Com o Historico do Cartão: [historico_cartao]
>> Tabela Com o Extrato da Conta Corrente: [extrato_cc]
>> Tabela Para Ser Inserido os Lançamentos previsto: [previsao_lancamento]


# Configuração de ambiente

In [17]:
# Essa configuração é para o datetime entender os meses como mês Brasil
locale.setlocale(locale.LC_ALL, 'pt_BR.utf8')

'pt_BR.utf8'

# Captura de Dados nas bases
### Trazendo para o jupyter os dados da base

In [18]:
acc = ConectarAccess(arquivo_base)

# Base do cartão de crédito
colunas_cartao=['Codigo','Cartao','Data_Compra','Lancamento','Tipo_Lancamento','Valor','Parcela','Lancada','Vencimento','Arquivo']
base_cartao = acc.select(f"""SELECT * FROM {tabela_cartao} """)
base_cartao = pd.DataFrame([[a,b,c,d,e,f,g,h,i,j] for (a,b,c,d,e,f,g,h,i,j) in base_cartao], 
                columns=colunas_cartao)

# Base da conta corrente
colunas_cc=['Codigo','Data Lancamento','Dependencia Origem','Historico','Tipo Historico','Data Balancete','Numero Documento','Valor','Mes Extrato']
base_cc = acc.select(f"""SELECT * FROM {tabela_cc} """)
base_cc = pd.DataFrame([[a,b,c,d,e,f,g,h,i] for (a,b,c,d,e,f,g,h,i) in base_cc], 
                columns=colunas_cc)

acc.close()

print("[INFO] Dados Capturados na Base de Informação")

[INFO] Dados Capturados na Base de Informação


### Verificando se a data de projeção é da fatura fechada ou abera

In [19]:
# Lista de ajuste do Tipo de Historico
vencimentos_aberto = base_cartao[base_cartao['Arquivo'].str.contains('Próxima')]['Vencimento'].unique()[0].date()

ultimo_vencimentos_fechado = max(
                                set(
                                    base_cartao[np.invert(
                                        base_cartao['Arquivo'].str.contains('Próxima').to_list())]\
                                    ['Vencimento'].unique()
                                    )
                                ).date()

print(f"Vencimento em aberto: {vencimentos_aberto}")
print(f"Ultimo Fatura Fechada: {ultimo_vencimentos_fechado}")

Vencimento em aberto: 2023-12-20
Ultimo Fatura Fechada: 2023-11-20


### Captura Data Frame com Compras Parceladas da ultima fatura

In [20]:
i = 0
list_lancamentos_compras_parceladas = []

lancamentos_compras_parceladas = base_cartao[base_cartao.Vencimento.astype(str) == str(ultimo_vencimentos_fechado)]

while len(lancamentos_compras_parceladas) != 0:
    if len(lancamentos_compras_parceladas) == 0:
        break

    i = i + 1
    lancamentos_compras_parceladas = base_cartao[base_cartao.Vencimento.astype(str) == str(ultimo_vencimentos_fechado)]
    lancamentos_compras_parceladas = lancamentos_compras_parceladas[lancamentos_compras_parceladas.Parcela != '']

    lancamentos_compras_parceladas['Parcela Atual'] = lancamentos_compras_parceladas.Parcela.str[:2].astype(int) + i
    lancamentos_compras_parceladas['Quantidade de Parcelas'] = lancamentos_compras_parceladas.Parcela.str[3:].astype(int)

    lancamentos_compras_parceladas = lancamentos_compras_parceladas[
            lancamentos_compras_parceladas['Parcela Atual'] <= 
            lancamentos_compras_parceladas['Quantidade de Parcelas']
        ]

    lancamentos_compras_parceladas['Metodo'] = "Compras Parceladas da última fatura"
    lancamentos_compras_parceladas['Valor'] = -1*lancamentos_compras_parceladas['Valor']
    lancamentos_compras_parceladas['Data_Base'] = ultimo_vencimentos_fechado + relativedelta(months=+i)
    lancamentos_compras_parceladas['Lancamento'] = lancamentos_compras_parceladas['Lancamento'] + \
                                                    " " + \
                                                    lancamentos_compras_parceladas['Parcela Atual'].astype(str) + \
                                                    "/" + \
                                                    lancamentos_compras_parceladas['Quantidade de Parcelas'].astype(str)

    #Renomeia as colunas
    lancamentos_compras_parceladas = lancamentos_compras_parceladas.rename(columns={"Data_Compra": "Data Lancamento", 
                                            "Lancamento": "Historico",
                                            "Tipo_Lancamento":"Tipo Historico",
                                            "Cartao":"Local_Lancado"})

    lancamentos_compras_parceladas['Data Lancamento'] = lancamentos_compras_parceladas['Data Lancamento'].apply(lambda x: x.date())

    # Seleciona as colunas
    lancamentos_compras_parceladas = lancamentos_compras_parceladas[[
        'Data_Base',
        'Data Lancamento',
        'Historico',
        'Tipo Historico',
        'Valor',
        'Local_Lancado',
        'Metodo'
    ]]

    list_lancamentos_compras_parceladas.append(lancamentos_compras_parceladas)

lancamentos_compras_parceladas = pd.concat(list_lancamentos_compras_parceladas)

### Captura Data Frame com Compras Recorrentes

In [21]:
list_lancamentos_recorrentes = []
for dt in [d for d in lancamentos_compras_parceladas.Data_Base.unique()]:
    lancamentos_recorrentes = base_cartao[base_cartao.Vencimento.astype(str) == str(ultimo_vencimentos_fechado)]

    list_recorrentes = ["SPOT", "NIVEL", "HBO", "TIM", "FGV"]
    list_r = []
    for i, row in lancamentos_recorrentes.iterrows():
        valida = False
        for t in list_recorrentes:
            if t in row.Lancamento:
                valida = True
        list_r.append(valida)

    lancamentos_recorrentes = lancamentos_recorrentes[list_r]

    lancamentos_recorrentes['Metodo'] = "Compras Recorrentes"
    lancamentos_recorrentes['Data_Base'] = dt
    lancamentos_recorrentes['Valor'] = -1*lancamentos_recorrentes['Valor']

    #Renomeia as colunas
    lancamentos_recorrentes = lancamentos_recorrentes.rename(columns={"Data_Compra": "Data Lancamento", 
                                            "Lancamento": "Historico",
                                            "Tipo_Lancamento":"Tipo Historico",
                                            "Cartao":"Local_Lancado"})

    # Seleciona as colunas
    lancamentos_recorrentes = lancamentos_recorrentes[[
        'Data Lancamento',
        'Historico',
        'Tipo Historico',
        'Valor',
        'Local_Lancado',
        'Metodo',
        'Data_Base'
    ]]

    list_lancamentos_recorrentes.append(lancamentos_recorrentes)

lancamentos_recorrentes = pd.concat(list_lancamentos_recorrentes)

### Captura Data Frame com compras já processadas na fatura aberta

In [22]:
lacamentos_processados = base_cartao[base_cartao.Vencimento.astype(str) == str(vencimentos_aberto)]

lacamentos_processados['Metodo'] = "Compras Processadas"
lacamentos_processados['Data_Base'] = vencimentos_aberto
lacamentos_processados['Valor'] = -1*lacamentos_processados['Valor']
#Renomeia as colunas
lacamentos_processados = lacamentos_processados.rename(columns={"Data_Compra": "Data Lancamento", 
                                        "Lancamento": "Historico",
                                        "Tipo_Lancamento":"Tipo Historico",
                                        "Cartao":"Local_Lancado"})
lacamentos_processados['Data Lancamento'] = lacamentos_processados['Data Lancamento'].apply(lambda x: x.date())

# Seleciona as colunas
lacamentos_processados = lacamentos_processados[[
    'Data Lancamento',
    'Historico',
    'Tipo Historico',
    'Valor',
    'Local_Lancado',
    'Metodo',
    'Data_Base'
]]

C:\Users\vinir\AppData\Local\Temp\ipykernel_16156\2055667222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lacamentos_processados['Metodo'] = "Compras Processadas"
C:\Users\vinir\AppData\Local\Temp\ipykernel_16156\2055667222.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lacamentos_processados['Data_Base'] = vencimentos_aberto
C:\Users\vinir\AppData\Local\Temp\ipykernel_16156\2055667222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

### Captura Data Frame com a ultima fatura fechada

In [23]:
lancamento_fatura_fechada = base_cartao[base_cartao.Vencimento.astype(str) == str(ultimo_vencimentos_fechado)]
lancamento_fatura_fechada['Metodo'] = "Ultma Fatura Fechada"
lancamento_fatura_fechada['Data_Base'] = ultimo_vencimentos_fechado
lancamento_fatura_fechada['Valor'] = -1*lancamento_fatura_fechada['Valor']

#Renomeia as colunas
lancamento_fatura_fechada = lancamento_fatura_fechada.rename(columns={"Data_Compra": "Data Lancamento", 
                                          "Lancamento": "Historico",
                                          "Tipo_Lancamento":"Tipo Historico",
                                          "Cartao":"Local_Lancado"})

# Seleciona as colunas
lancamento_fatura_fechada = lancamento_fatura_fechada[[
    'Data Lancamento',
    'Historico',
    'Tipo Historico',
    'Valor',
    'Local_Lancado',
    'Metodo',
    'Data_Base'
]]

C:\Users\vinir\AppData\Local\Temp\ipykernel_16156\3929059510.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lancamento_fatura_fechada['Metodo'] = "Ultma Fatura Fechada"
C:\Users\vinir\AppData\Local\Temp\ipykernel_16156\3929059510.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lancamento_fatura_fechada['Data_Base'] = ultimo_vencimentos_fechado
C:\Users\vinir\AppData\Local\Temp\ipykernel_16156\3929059510.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

### Captura Data Frame com valores lançados em conta correntes

In [24]:
list_lancamentos_cc = []
lista_data = [d for d in lancamentos_compras_parceladas.Data_Base.unique()]
lista_data.append(ultimo_vencimentos_fechado)
for dt in lista_data:
    base_cc['Vencimento Cartao'] = base_cc['Mes Extrato'].apply(lambda x: date(int(x[:4]),int(x[4:]),20))
    lancamentos_cc = base_cc[base_cc['Vencimento Cartao'] == ultimo_vencimentos_fechado + relativedelta(months=-1)]
    list_tipos_lancamento = ['Casa', 'Emprestimo', 'Educação', 'Pagamento BB']
    list_r = []
    for i, row in lancamentos_cc.iterrows():
        if row['Tipo Historico'] in  list_tipos_lancamento:
            list_r.append(True)
        else:
            list_r.append(False)

    lancamentos_cc = lancamentos_cc[list_r]
    lancamentos_cc['Local_Lancado'] = 'Conta Corrente'
    lancamentos_cc['Metodo'] = 'Lancamentos Recorrentes em Conta Corrente'
    lancamentos_cc['Data_Base'] = dt

    #Renomeia as colunas
    lancamentos_cc = lancamentos_cc.rename(columns={"Data_Compra": "Data Lancamento", 
                                            "Lancamento": "Historico",
                                            "Tipo_Lancamento":"Tipo Historico",
                                            "Vencimento Cartao":"Vencimento"})

    # Seleciona as colunas
    lancamentos_cc = lancamentos_cc[[
        'Data Lancamento',
        'Historico',
        'Tipo Historico',
        'Valor',
        'Local_Lancado',
        'Metodo',
        'Data_Base'
    ]]

    list_lancamentos_cc.append(lancamentos_cc)
lancamentos_cc = pd.concat(list_lancamentos_cc)

# Ajustes
### Ajuste de compras parceladas já lançadas

In [25]:
proxima_compras_parceladas = lancamentos_compras_parceladas\
                        [lancamentos_compras_parceladas.Data_Base == vencimentos_aberto]

list_proxima_compras_parceladas = []
for i,row_pc in proxima_compras_parceladas.iterrows():
    verifica = True
    for i,row_pr in lacamentos_processados.iterrows():
        bol_filter = (abs(abs(row_pc.Valor) - abs(row_pr.Valor)) < 0.1) and\
                    (row_pc['Data Lancamento'].day == row_pr['Data Lancamento'].day)
        if bol_filter:
            verifica = False
    list_proxima_compras_parceladas.append(verifica)

parcelas_nao_processadas = proxima_compras_parceladas[list_proxima_compras_parceladas]
lancamentos_compras_parceladas = pd.concat([lancamentos_compras_parceladas\
                        [lancamentos_compras_parceladas.Data_Base != vencimentos_aberto],
                        parcelas_nao_processadas])


# Concatena os Data Frames

In [26]:
df_final = pd.concat([lancamento_fatura_fechada,
                        lancamentos_cc,
                        lancamentos_compras_parceladas,
                        lancamentos_recorrentes,
                        lacamentos_processados
                    ])
    
#Renomeia as colunas
df_final = df_final.rename(columns={"Data Lancamento":"Data_Lancamento",
                                    "Tipo Historico":"Tipo_Historico",})

df_final.Valor = df_final.Valor.apply(lambda x: round(x,2))

# Carrega dados na base

In [27]:
# Instancia o objeto que faz as conexões com a base de dados
acc = ConectarAccess(arquivo_base)

print("[INFOR] Deletando dados na tabela de projeção")
acc.delete(f"DELETE FROM {tabela_previsao}")

acc.insertAcc(df_final, tabela_previsao)
print('[INFO] Dados Inseridos com sucesso na tabela de previsão')

# Fecha a conexão
acc.close()

[INFOR] Deletando dados na tabela de projeção
[INFO] Dados Inseridos com sucesso na tabela de previsão
